In [82]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [83]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
%matplotlib inline

In [84]:
def region_of_interest(img, vertices):
    ##"""
    ##Applies an image mask.
    
    ##Only keeps the region of the image defined by the polygon
    ##formed from `vertices`. The rest of the image is set to black.
    ##"""
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [85]:
def convert_to_gscale(image):
    img_grayscale = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY) 
    return img_grayscale

In [86]:
def apply_canny(image,thresh =(125,200)):
    img_canny = cv2.Canny(image,thresh[0],thresh[1])
    return img_canny

In [87]:
def apply_gaussblur(image, kernel_size = 3):
    img_blur = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    return img_blur

In [88]:
def apply_hough(image):
        
    lines = cv2.HoughLinesP(image, 2, np.pi/180, 15, np.array([]), 10, 20)
    return lines

In [89]:
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [90]:
def draw_lines(image, lines, color=[0, 255, 0], thickness=3):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    #for line in lines:
    #    for x1,y1,x2,y2 in line:
    #        cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
    img_copy = np.copy(image)
    
    x_center = (np.rint(image.shape[1]/2)).astype(int)
                
    leftfit_xarr = []
    leftfit_yarr = []
    
    rightfit_xarr = []
    rightfit_yarr = []
    
    slope_thresh1 = -0.5
    slope_thresh2 = 0.5
    #print(type(lines),lines.shape)
    
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            #few error correction cases - eliminate slope of 0 or infinity
            if ((x1 == x2) or (y1 == y2)):
                break;  
            
            #check slope and classify them as left and right lines
            slope = (y2-y1)/(x2-x1);
            
            #print('Slope is :',slope, x1, x2, x_center)
            if ((x1 < x_center) and (x2 < x_center) and (slope < slope_thresh1) ):
                #print(slope, x1,y1,x2,y2,x_center)
                leftfit_xarr = np.append(leftfit_xarr, x1)
                leftfit_yarr = np.append(leftfit_yarr,y1)
                leftfit_xarr = np.append(leftfit_xarr, x2)
                leftfit_yarr = np.append(leftfit_yarr,y2)
                                
            elif ((x1 > x_center) and (x2 > x_center) and (slope > slope_thresh2)):
                rightfit_xarr = np.append(rightfit_xarr, x1)
                rightfit_yarr = np.append(rightfit_yarr,y1)
                rightfit_xarr = np.append(rightfit_xarr, x2)
                rightfit_yarr = np.append(rightfit_yarr,y2)

    #fit the x and y lane coordinates collected 
    fitleft_m, fitleft_c = np.polyfit(leftfit_xarr, leftfit_yarr, 1)
    fitright_m, fitright_c = np.polyfit(rightfit_xarr, rightfit_yarr, 1)
    
    #define the ycorodinates where the linear fit will be evaluated
    y_vals_max = img_copy.shape[0]
    y_vals_min = img_copy.shape[0] - 210
    
    #calculate x values of left and right lanes from the linear fit
    x_vals_lft_min = ((y_vals_min - fitleft_c)/fitleft_m).astype(int)
    x_vals_lft_max = ((y_vals_max - fitleft_c)/fitleft_m).astype(int)
    
    x_vals_rt_min = ((y_vals_min - fitright_c)/fitright_m).astype(int)
    x_vals_rt_max = ((y_vals_max - fitright_c)/fitright_m).astype(int)
    
    #draw the line on the source image
    cv2.line(image, (x_vals_lft_min, y_vals_min), (x_vals_lft_max, y_vals_max), color, thickness)
    cv2.line(image, (x_vals_rt_min, y_vals_min), (x_vals_rt_max, y_vals_max), color, thickness)
    
    return image

In [91]:
# function that shows outputlines from the hough transform - for debusgging only
def show_houghlines(image, lines, color = [255,0,0], thickness = 3):
    img_copy = np.copy(image)
    for line in lines:
        for x1, y1, x2, y2 in line:
           cv2.line(img_copy, (x1, y1), (x2, y2), color, thickness) 
    return 

In [92]:
def colormask_yellowwhite(image):
    
    # Add yellow color mask and white mask to better detect the lanes
    # White horizontal marker lanes seems to introduce errors int the pipeline 
        
    #contains actual image on the left    
    image_left = np.copy(image)
    image_rt = np.copy(image)
    
    #Look for white pixels
    white_threshold = 200
    lower_white = np.array([200, 200, 200])
    upper_white = np.array([255, 255, 255])
    white_mask = cv2.inRange(image_rt, lower_white, upper_white)
    white_image = cv2.bitwise_and(image_rt, image_rt, mask=white_mask)
  
    # Look for yellow pixels
    hsv = cv2.cvtColor(image_left, cv2.COLOR_BGR2HSV)
    low_yellow = np.array([90,100,100])
    up_yellow = np.array([110,255,255])
        
    yellow_mask = cv2.inRange(hsv, low_yellow, up_yellow)
    yellow_image = cv2.bitwise_and(image_left, image_left, mask=yellow_mask)
  
    # Combine the two above images
    image_clrmasked = cv2.addWeighted(white_image, 1., yellow_image, 1., 0.)
    
    return image_clrmasked

In [98]:
#Image pipeline 
def image_pipeline(image, canny_thresh, vertices):  
    mask_img = colormask_yellowwhite(image)
    img_grayscale = convert_to_gscale(mask_img)
    blur_img = apply_gaussblur(img_grayscale)
    img_canny = apply_canny(blur_img,canny_thresh)
    img_bounded = region_of_interest(img_canny, vertices)
    lines = apply_hough(img_bounded)
    #show_houghlines(img_bounded, lines) 
    filled_image = draw_lines(image, lines)
    #ant_image = anotate_image(filled_image)
    
    return filled_image

In [94]:
canny_thresh = (50,150)
vertices = np.array([[0,540],[450,320],[500,320],[960,540]],dtype=np.int32);

In [95]:
### Process test images through the pipeline

img_folder = './test_images/'
output_folder = './test_images_output/'
img_list = ['solidWhiteCurve.jpg', 'solidWhiteRight.jpg', 'solidYellowCurve.jpg', 'solidYellowCurve2.jpg',
                'solidYellowLeft.jpg','whiteCarLaneSwitch.jpg']

for img_name in img_list:
    img_path = img_folder + img_name
    image = mpimg.imread(img_path)
    image_pipeline(image, canny_thresh, [vertices])
    mpimg.imsave(output_folder + 'out_' + img_name, image)        

In [96]:
### Process video
# Import everything needed to edit/save/watch video clips
import moviepy
from moviepy.editor import *
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = './test_videos_output/white.mp4'
clip = VideoFileClip("./test_videos/solidWhiteRight.mp4")
output_frames = [image_pipeline(frame, canny_thresh, [vertices]) for frame in clip.iter_frames()]
white_clip = ImageSequenceClip(output_frames, fps=clip.fps)
%time white_clip.write_videofile(white_output, audio=False)
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

[MoviePy] >>>> Building video ./test_videos_output/white.mp4
[MoviePy] Writing video ./test_videos_output/white.mp4


100%|██████████| 221/221 [00:04<00:00, 46.77it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./test_videos_output/white.mp4 

CPU times: user 219 ms, sys: 181 ms, total: 401 ms
Wall time: 5.77 s


In [97]:
yellow_output = './test_videos_output/yellow.mp4'
clip1 = VideoFileClip("./test_videos/solidYellowLeft.mp4")
count = 1
output_frames = []

#for frame in clip1.iter_frames():    
#    output_frames = np.append(output_frames,image_pipeline(frame, canny_thresh, [vertices], count))
#    count = count+1

output_frames = [image_pipeline(frame, canny_thresh, [vertices]) for frame in clip1.iter_frames()]    
yellow_clip = ImageSequenceClip(output_frames, fps=clip.fps)
%time yellow_clip.write_videofile(yellow_output, audio=False)
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

[MoviePy] >>>> Building video ./test_videos_output/yellow.mp4
[MoviePy] Writing video ./test_videos_output/yellow.mp4


100%|██████████| 681/681 [00:19<00:00, 34.92it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./test_videos_output/yellow.mp4 

CPU times: user 854 ms, sys: 1.53 s, total: 2.38 s
Wall time: 20.6 s
